In [1]:
using Statistics
using TextAnalysis

# ファイルを読み込む
file_path = "./data/20231102_alice.txt"
text = read(file_path, String)

# テキストを小文字に変換
text = lowercase(text)
doc = StringDocument(text)
# prepare!(doc, strip_punctuation)
# テキストを文に分割する

A StringDocument{String}
 * Language: Languages.English()
 * Title: Untitled Document
 * Author: Unknown Author
 * Timestamp: Unknown Time
 * Snippet: alice was beginning to get very tired of sitting b

In [2]:
include("./lib/expand_contractions.jl")
text = expand_contractions(text)

"alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, “and what is the use of a book,” thought alice “wi" ⋯ 148192 bytes ⋯ "yes bright and eager with many a strange tale, perhaps even with the dream of wonderland of long ago: and how she would feel with all their simple sorrows, and find a pleasure in all their simple joys, remembering her own child life, and the happy summer days.\n\nthe end"

In [3]:
sentences = split(text, r"[.?!“”(),\n]+")

4861-element Vector{SubString{String}}:
 "alice was beginning to get very tired of sitting by her sister on the bank"
 " and of having nothing to do: o" ⋯ 33 bytes ⋯ "the book her sister was reading"
 " but it had no pictures or conversations in it"
 " "
 "and what is the use of a book"
 " thought alice "
 "without pictures or conversations"
 "so she was considering in her own mind "
 "as well as she could"
 " for the hot day made her feel very sleepy and stupid"
 ⋮
 " be herself a grown woman  and how she would keep"
 " through all her riper years"
 " the simple and loving heart of" ⋯ 41 bytes ⋯ "about her other little children"
 " and make their eyes bright and eager with many a strange tale"
 " perhaps even with the dream of" ⋯ 46 bytes ⋯ "l with all their simple sorrows"
 " and find a pleasure in all their simple joys"
 " remembering her own child life"
 " and the happy summer days"
 "the end"

In [4]:

# 各文を単語に分割し、STARTとENDを追加する
processed_sentences :: Vector{Vector{String}} = []
for sentence in sentences
    words :: Vector{String} = split(sentence)
    if isempty(words) || length(words) < 3
        continue
    end
    pushfirst!(words, "START")
    push!(words, "END")
    push!(processed_sentences, words :: Vector{String})
end

In [5]:
include("./lib/probability.jl")

calculator = TextEntropyCalculator(processed_sentences)
entropy = calculate_entropy(calculator, "START is the rabbit on the table white END")
println("Cross Entropy: ", entropy)

Calculating bigrams and singlegrams...total_bigrams: 29115
total_singlegrams: 32592
Cross Entropy: 

(3.5704825817955212, Any[((

"START", "START"), 0.05406032139246206), (("START", "is"), 0.007827195905091443), (("is", "the"), 0.07282629434072282), (("the", "rabbit"), 0.0007210358370152185), (("rabbit", "on"), 0.36450956506752735), (("on", "the"), 0.04664928334017399), (("the", "table"), 0.00026080019636720667), (("table", "white"), 0.059283765052412715), (("white", "END"), 0.05334131075110456)])


In [6]:
p_sentences = []

for sentence in processed_sentences
    push!(p_sentences, join(sentence, " "))
end

processed_text = join(p_sentences, "\n")

output = "./data/alice_processed.txt"
open(output, "w") do f
    write(f, processed_text)
end

162772

In [7]:
include("./lib/calc_entropy.jl")
include("./lib/probability.jl")
include("./lib/biagram_export.jl")

# 指定された文
case1 = ["START is the rabbit on the table white END", "START does the rabbit on the table red END"]
case2 = ["START is the girl reading END", "START is the rabbit reading END"]
case3 = ["START is the caterpillar tired END", "START does the caterpillar tired END"]
case4 = ["START is the house old END", "does the house old END"]
e1_entropy, e1_words = calculate_entropy(calculator, case1[1])
generate_bigram_graph([e1_words, e1_entropy], "sample_graph")



function compare_2sentences(sentence1, sentence2, num)
    words1 = split(sentence1)
    words2 = split(sentence2)
    e1_entropy, e1_words = calculate_entropy(calculator, sentence1)
    e2_entropy, e2_words = calculate_entropy(calculator, sentence2)
    println("Cross Entropy: ", e1_entropy, " ", e2_entropy)
    generate_bigram_graph([e1_words, e1_entropy], "graph-$num-1")
    generate_bigram_graph([e2_words, e2_entropy], "graph-$num-2")
end
compare_2sentences(case1[1], case1[2], 1)
compare_2sentences(case2[1], case2[2], 2)
compare_2sentences(case3[1], case3[2], 3)
compare_2sentences(case4[1], case4[2], 4)

Cross Entropy: 3.5704825817955212 3.699953213805839
Cross Entropy: 3.739122627703332 3.5846182505516415


Cross Entropy: 4.314047807237659 4.394018952818498
Cross Entropy: 3.386262105341753 3.6027394808583324


"/workspaces/JuliaAliceBigram/graph-4-2.png"